In [1]:
!pip install --quiet -U sentence-transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [3]:
# Load job titles data 
titles_df = pd.read_csv("/kaggle/input/df-dropna-lightcast-titles/lightcast-titles.csv")
titles = titles_df["Title"].tolist()

candidates_df = pd.read_excel('/kaggle/input/df-dropna-lightcast-titles/df-dropna.xlsx')

# Assuming 'df_candidates' and 'df_titles' are your DataFrames
# Encode candidate profiles, designation
candidate_profiles = candidates_df["candidate_profiles"].tolist()
jd_designations = candidates_df["designation"].tolist()

candidate_embeds = model.encode(candidate_profiles,show_progress_bar=False)
designation_embeds = model.encode(jd_designations,show_progress_bar=False)

# Encode titles dataset  
title_embeds = model.encode(titles, show_progress_bar=False)

In [4]:
# Cosine Similarity
candidate_cosine_sim = cosine_similarity(candidate_embeds, title_embeds)
jd_cosine_sim = cosine_similarity(designation_embeds,title_embeds)

# Get the index of the closest matches
cosine_best_match_candidate = candidate_cosine_sim.argmax(axis=1)
cosine_best_match_designation = jd_cosine_sim.argmax(axis=1)

In [5]:
# Get best matched titles
matched_titles_candidates = [titles[idx] for idx in cosine_best_match_candidate]
matched_titles_designation = [titles[idx] for idx in cosine_best_match_designation]


# Add to df
candidates_df['norm_profile'] = matched_titles_candidates
candidates_df['norm_designation'] = matched_titles_designation

In [6]:
# Reorder columns
columns_reordered = ['status', 'experience', 'present_ctc', 'expected_ctc', 'notice_period',
                     'candidate_profiles','norm_profile', 'job_categories', 
                     'designation', 'norm_designation','Relevant', 'exp_min', 'exp_max', 'ctc_fixed_min', 
                     'ctc_fixed_max']

# Reindex the DataFrame with the reordered columns
candidates_df = candidates_df.reindex(columns=columns_reordered)

candidates_df.head()  # Displaying the reordered DataFrame

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
0,ACCEPTED,2.00,3.6,4.14,15,Senior Sales Executive,Sales Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,1,0.5,3.0,3.0,5.0
1,ACCEPTED,11.00,3.0,4.00,1,Marketing executive,Marketing Executive,"Inside Sales (Outbound),Sales",Inside Sales,Inside Sales Professional,1,0.5,3.0,3.0,5.0
2,ACCEPTED,2.50,5.0,5.75,15,Android Developer,Android Developer,Frontend Developer,React Native Developer,React Native Developer,1,2.0,4.0,6.0,10.0
3,ACCEPTED,4.08,7.0,8.05,15,Android Mobile Application Developer|||Android...,React Native Developer,Frontend Developer,React Native Developer,React Native Developer,1,2.0,4.0,6.0,10.0
4,ACCEPTED,3.50,4.0,8.00,1,Devops engineer,DevOps Engineer,Developer Operations,Developer operations,Operations Developer,1,1.0,5.0,6.0,10.0


In [7]:
candidates_df.to_excel("all-miniLM-L12-v2_norm-dataset.xlsx", index=False)

In [8]:
candidates_df.sample(10)

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,norm_profile,job_categories,designation,norm_designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
1612,ACCEPTED,10.00,15.0,18.75,30,Key Account Manager,Key Account Manager,Key account manager/ Customer success manager,Growth strategist - For Amazon,Growth Strategist,1,4.0,8.0,10.0,16.0
8975,ACCEPTED,2.00,12.5,18.00,45,Software Engineer 1,Software Engineer,Backend Developer,SDE I - Backend,Back End Developer,1,1.0,3.0,20.0,25.0
3753,ACCEPTED,0.60,5.0,6.50,30,business development associate(Business Develo...,Business Development Associate,Business development,Business Development Executive (Mumbai),Business Development Executive,1,0.5,3.0,3.0,5.5
13495,REJECTED,2.08,4.0,4.60,15,Account Executive,Account Executive,Inside Sales (Outbound),Project Manager,Project Manager,0,0.5,5.0,4.0,6.5
10417,ACCEPTED,0.10,1.4,1.80,1,loan consultant,Loan Consultant,"Business development,Finance,Lead generation,M...",Business Development Executives(Filed Sales-Loan),Business Development Sales Executive,1,1.0,4.0,1.2,2.4
3055,ACCEPTED,1.00,4.2,0.00,0,business development executive,Business Development Executive,Business development,Business Development Executive - Hindi,Business Development Executive,1,0.5,3.0,3.0,5.5
2482,ACCEPTED,4.42,7.2,8.28,60,Mean Stack Developer|||Software Associate,Mean Stack Developer,Full stack developer,Nodejs Developer (Nodejs+Angular),AngularJS Developer,1,4.0,6.0,10.0,14.0
3397,ACCEPTED,0.60,5.0,0.00,0,business development associate(Business Develo...,Business Development Associate,Business development,Business Development Executive - Hindi,Business Development Executive,1,0.5,3.0,3.0,5.5
2617,ACCEPTED,4.00,4.0,4.60,30,Editorial Assistant|||Social Media and Editori...,Social Media Executive,Social media marketing (Organic),Digital marketing manager,Digital Marketing Manager,1,1.5,5.0,3.5,5.5
8687,ACCEPTED,3.00,8.0,15.00,17,Junior Software Developer,Student Software Developer,Backend Developer,Backend developer,Back End Developer,1,3.0,5.0,25.0,30.0
